In [25]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pylab as plt
import datetime
from scipy import integrate
from tick.hawkes import *
from scipy.optimize import *
from tick.plot import plot_hawkes_kernels
from scipy import stats
from tick.hawkes import SimuHawkes, HawkesKernelPowerLaw, HawkesConditionalLaw, HawkesKernel0
import statsmodels.api as sm
import itertools

%matplotlib inline

In [26]:
def csv_reader(name):
    df = pd.read_csv(name,header=None)

    df.set_index(0)

    df.drop(df.columns[0], axis = 1, inplace=True)
    df.columns = df.iloc[0]
    df = df.iloc[1:]

    df["Bid_time"] = df["Bid_time"].astype(float)
    df["Mid_IV"] = df["Mid_IV"].astype(float)
    df["Mid_price"] = df["Mid_price"].astype(float)
    return df
    

In [27]:
def single_exp(decays, events):
        return - HawkesExpKern(decays=decays[0], penalty='elasticnet', tol=1e-8,
                          elastic_net_ratio=0.9, max_iter=1000).fit(events).score()

In [28]:
def sum_exp(decays, events):
        return - HawkesSumExpKern(decays=decays, penalty='elasticnet', tol=1e-8,
                          elastic_net_ratio=0.9, max_iter=1000).fit(events).score()

In [29]:
def sum_3exp_minimiser(x, timestamps):
    return minimize(sum_exp, x0 = [x]*3, args = (timestamps), method = 'Nelder-Mead', tol =1e-5)

In [30]:
def sum_2exp_minimiser(x, timestamps):
    return minimize(sum_exp, x0 = [x] * 2, args = (timestamps), method = 'Nelder-Mead', tol =1e-5)

In [31]:
def exp_minimiser(x,timestamps):
    return minimize(single_exp, x0 = [x], args = (timestamps), method = 'Nelder-Mead', tol =1e-5)

In [33]:
def grid_search(fun, start, stop, tol, timestamps, init = -np.inf, init_betas = 0):
    
    N = 20
    
    grid = np.linspace(start,stop, N)
    print(f"Grid: {start} : {stop}")
    best_result = init
    best_betas = init_betas

    for i in range(N):
        try:
            optimised = fun(grid[i],timestamps)
            result = -optimised.fun 
            betas = optimised.x
            if result > best_result:
                best_betas = betas
                best_initial_value = grid[i]
                if result - best_result < tol:
                    return best_betas
                best_result = result
        except:
            print("Iteration {} erred".format(i))
            
    #sometimes it doesn't improve at all during the search, so some variables might not be defined
    try:
        next_start = best_initial_value*0.8
        next_stop = best_initial_value*1.2
        print(f"Best value so far: {best_result} found at {best_initial_value}")
        print(f"Optimal betas so far: {best_betas}")
        return grid_search(fun, next_start, next_stop, tol, timestamps, init = best_result, init_betas = best_betas)
    
    except:
        return best_betas   
    

In [34]:
def resid(x, intensities, timestamps, dim, method):
    print(dim)
    arrivals = timestamps[dim]
    thetas = np.zeros(len(arrivals) - 1)
    ints = intensities[dim]
    for i in range(1, len(arrivals)):
        mask = (x <= arrivals[i]) & (x >= arrivals[i - 1])
        xs = x[mask]
        ys = ints[mask]
        try:
            thetas[i - 1] = method(ys, xs)
        except:
            thetas[i - 1] = np.nan

    return thetas

def goodness_of_fit_par(learner, arrivals, step, method):
    dimension = learner.n_nodes
    intensities = learner.estimated_intensity(arrivals, step)[0]
    x = learner.estimated_intensity(arrivals, step)[1]
    residuals = [resid(x, intensities, arrivals, dim, method) for dim in range(dimension)]
    return residuals

def ks_test(resid):
    for res in resid:
        print(stats.kstest(res[np.logical_not(np.isnan(res))], 'expon'))

def plot_resid(resid, rows, cols):
    fig, axes = plt.subplots(nrows=rows, ncols=cols)
    fig.subplots_adjust(hspace=0.5)
    fig.suptitle('Goodness-of-fit for nonparametric HP')

    for ax, res in zip(axes, resid):
        k = stats.probplot(res, dist=stats.expon, fit=True, plot=ax, rvalue=False)
        ax.plot(k[0][0], k[0][0], 'k--')

def ks_test(resid):
    return [
        stats.kstest(res[np.logical_not(np.isnan(res))], 'expon')
        for res in resid
    ]

def lb_test(resid):
    return [
        sm.stats.acorr_ljungbox(res[np.logical_not(np.isnan(res))], lags=[3], return_df=True)
        for res in resid
    ]

def ed_test(resid):
    results = []
    for res in resid:
        res_ = res[np.logical_not(np.isnan(res))]
        results.append(
            np.sqrt(len(res_)) * (np.var(res_, ddof=1) - 1) / np.sqrt(8)
        )
    return results

In [195]:
def goodness_of_fit_nonpar(nplearner, arrivals, timestep, method):
    # setup simulation object
    hp = SimuHawkes(baseline=nplearner.baseline)
    # set kernels
    support = np.arange(0, np.max(nplearner.get_kernel_supports()), timestep)

    for i, j in itertools.product(range(nplearner.n_nodes), repeat=2):
        print('Kernel {} set.'.format([i, j]))
        y = nplearner.get_kernel_values(i, j, support)
        kernel = HawkesKernelTimeFunc(t_values=support, y_values=y)
        hp.set_kernel(i, j, kernel)

    hp.track_intensity(timestep)
    hp.set_timestamps(timestamps=arrivals)
    dimension = hp.n_nodes
    intensities = hp.tracked_intensity
    x = hp.intensity_tracked_times
    residuals = [resid(x, intensities, arrivals, dim, method) for dim in range(dimension)]
    return residuals

# AAPL Data

In [247]:
# session = "AM"
# session = "MID"
session = "PM"

n_minus_ts_ticks_25dc = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_60dc.csv')
n_minus_ts_ticks_25dp = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_60dp.csv')
n_minus_ts_ticks_50dc = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_50dc.csv')

n_plus_ts_ticks_25dc = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_60dc.csv')
n_plus_ts_ticks_25dp = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_60dp.csv')
n_plus_ts_ticks_50dc = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_50dc.csv')

In [248]:
_cutoff = 1200

all_data = [
            n_plus_ts_ticks_25dc[:_cutoff], n_minus_ts_ticks_25dc[:_cutoff],
            n_plus_ts_ticks_50dc[:_cutoff], n_minus_ts_ticks_50dc[:_cutoff],
            n_plus_ts_ticks_25dp[:_cutoff], n_minus_ts_ticks_25dp[:_cutoff],
]

all_timestamps = [
    np.array(list(df['Bid_time']))
    for df in all_data
]
    
max_min_time = max([min(ts_df) for ts_df in all_timestamps])
min_max_time = min([max(ts_df) for ts_df in all_timestamps])

trimmed_ts = [
    ts[(max_min_time < ts) & (ts < min_max_time)] - max_min_time
    for ts in all_timestamps
]

for ts_ in trimmed_ts:
    print(ts_[0])
    print(ts_[-1])
    print("Increasing: {}".format(np.all(np.diff(ts_) > 0)))

print()
print("Before : {}".format(max([len(ts) for ts in all_timestamps])))
print("After : {}".format(max([len(ts) for ts in trimmed_ts])))
all_timestamps = trimmed_ts

7.0
113176.0
Increasing: True
4.0
113141.0
Increasing: True
57.0
113928.0
Increasing: True
30.0
113938.0
Increasing: True
11.0
113908.0
Increasing: True
10.0
114012.0
Increasing: True

Before : 1200
After : 1185


# Single Expo

In [155]:
start = 0.015
stop = 1.2

single_expo_best_beta = grid_search(exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=all_timestamps)

Grid: 0.015 : 1.2


/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/tick/solver/agd.py:155: RuntimeWarning: invalid value encountered in multiply
  y[:] = x + (prev_t - 1) / t * (x - prev_x)


Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Best value so far: 0.02817067521092699 found at 1.1376315789473683
Optimal betas so far: [0.11373538]
Grid: 0.9101052631578948 : 1.365157894736842
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0


In [157]:
BEST_BETA = single_expo_best_beta[0]

single_expo_learner = HawkesExpKern(decays=BEST_BETA,tol=1e-10, penalty='elasticnet', 
                          elastic_net_ratio=0.9, max_iter=20000)

single_expo_learner.fit(all_timestamps)
residuals = goodness_of_fit_par(single_expo_learner, all_timestamps, 1, integrate.trapz)

0
1
2
3
4
5


In [158]:
alphas = single_expo_learner.adjacency
betas = single_expo_learner.decays
baseline = single_expo_learner.baseline
loglik = single_expo_learner.score()

print("Alphas: ")
print(np.round(alphas, 2))

print("\n Baseline: ")
print(baseline)

table = {
    "Endogeneity": [max(np.linalg.eigvals(alphas))],
    "Likelihood": [loglik],
    "Max KS Test": max([ks.pvalue for ks in ks_test(residuals)]),
    "Max LB Test": max([list(lb.lb_pvalue)[0] for lb in lb_test(residuals)]),
    "Min ED Statistic": min(ed_test(residuals))
}

df = pd.DataFrame(table)
df

Alphas: 
[[0.   0.29 0.   0.04 0.06 0.01]
 [0.24 0.   0.05 0.02 0.05 0.04]
 [0.04 0.04 0.   0.28 0.11 0.05]
 [0.   0.   0.32 0.   0.1  0.08]
 [0.02 0.01 0.13 0.09 0.06 0.41]
 [0.03 0.03 0.11 0.09 0.49 0.  ]]

 Baseline: 
[0.00285605 0.00283443 0.00297024 0.00295919 0.0038731  0.00322903]


,Endogeneity,Likelihood,Max KS Test,Max LB Test,Min ED Statistic
0,0.614708,0.028172,0.650617,0.355427,0.061271


# Multi

In [187]:
start = 0.02
stop = 1.2

best_set_of_betas = grid_search(sum_3exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=all_timestamps)

Grid: 0.02 : 1.2


/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/tick/solver/agd.py:144: RuntimeWarning: overflow encountered in multiply
  + 1. / (2 * step) * norm(x - y) ** 2
/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/tick/solver/agd.py:144: RuntimeWarning: overflow encountered in double_scalars
  + 1. / (2 * step) * norm(x - y) ** 2


Step equals 0... at 0


/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/tick/solver/agd.py:155: RuntimeWarning: invalid value encountered in multiply
  y[:] = x + (prev_t - 1) / t * (x - prev_x)


Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equal

In [188]:
BEST_BETA = best_set_of_betas

learner = HawkesSumExpKern(decays=BEST_BETA,tol=1e-15, penalty='elasticnet', 
                          elastic_net_ratio=0.99, max_iter=20000)

learner.fit(all_timestamps)
residuals = goodness_of_fit_par(learner, all_timestamps, 1, integrate.trapz)

0
1
2
3
4
5


In [189]:
BEST_BETA

array([0.51174825, 0.18336858, 0.02221293])

In [190]:
alphas = learner.adjacency
betas = learner.decays
baseline = learner.baseline
loglik = learner.score()

print("Alphas: ")
print(np.round(alphas, 2))


print("Alphas: ")
print(np.round(betas, 2))

print("\n Baseline: ")
print(baseline)

table = {
    "Endogeneity": [max(np.linalg.eigvals(alphas.sum(-1)))],
    "Likelihood": [loglik],
    "Max KS Test": max([ks.pvalue for ks in ks_test(residuals)]),
    "Max LB Test": max([list(lb.lb_pvalue)[0] for lb in lb_test(residuals)]),
    "Min ED Statistic": min(ed_test(residuals))
}

df = pd.DataFrame(table)
df

Alphas: 
[[[0.   0.   0.  ]
  [0.   0.17 0.15]
  [0.   0.   0.  ]
  [0.   0.03 0.  ]
  [0.   0.02 0.06]
  [0.   0.   0.03]]

 [[0.   0.14 0.12]
  [0.   0.   0.  ]
  [0.   0.03 0.  ]
  [0.   0.01 0.  ]
  [0.   0.01 0.1 ]
  [0.   0.02 0.01]]

 [[0.   0.03 0.  ]
  [0.   0.03 0.  ]
  [0.   0.   0.  ]
  [0.   0.18 0.17]
  [0.   0.08 0.02]
  [0.   0.03 0.01]]

 [[0.   0.   0.  ]
  [0.   0.   0.  ]
  [0.   0.22 0.16]
  [0.   0.   0.  ]
  [0.   0.08 0.02]
  [0.   0.06 0.  ]]

 [[0.   0.   0.  ]
  [0.   0.   0.  ]
  [0.   0.1  0.  ]
  [0.   0.09 0.  ]
  [0.   0.06 0.  ]
  [0.   0.26 0.26]]

 [[0.   0.02 0.  ]
  [0.   0.02 0.  ]
  [0.   0.09 0.  ]
  [0.   0.08 0.  ]
  [0.   0.35 0.2 ]
  [0.   0.   0.  ]]]
Alphas: 
[0.51 0.18 0.02]

 Baseline: 
[0.00233588 0.0023421  0.0026151  0.00271351 0.00323827 0.00285923]


,Endogeneity,Likelihood,Max KS Test,Max LB Test,Min ED Statistic
0,0.671783+0.000000j,0.028923,0.755719,0.72964,-1.000444


# Sum of 2

In [98]:
start = 0.005
stop = 0.5

best_set_of_betas_2 = grid_search(sum_2exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=all_timestamps)

Grid: 0.005 : 0.5
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0.

In [99]:
BEST_BETA = best_set_of_betas_2

learner = HawkesSumExpKern(decays=BEST_BETA,tol=1e-10, penalty='elasticnet', 
                          elastic_net_ratio=0.9, max_iter=2000)

learner.fit(all_timestamps)
residuals = goodness_of_fit_par(learner, all_timestamps, 1, integrate.trapz)

0
1
2
3
4
5


In [100]:
alphas = learner.adjacency
betas = learner.decays
baseline = learner.baseline
loglik = learner.score()

print("Alphas: ")
print(np.round(alphas, 2))


print("Alphas: ")
print(np.round(betas, 2))

print("\n Baseline: ")
print(baseline)

table = {
    "Endogeneity": [max(np.linalg.eigvals(alphas.sum(-1)))],
    "Likelihood": [loglik],
    "Max KS Test": max([ks.pvalue for ks in ks_test(residuals)]),
    "Max LB Test": max([list(lb.lb_pvalue)[0] for lb in lb_test(residuals)]),
    "Min ED Statistic": min(ed_test(residuals))
}

df = pd.DataFrame(table)
df

Alphas: 
[[[0.   0.07]
  [0.11 0.37]
  [0.   0.01]
  [0.   0.  ]
  [0.   0.  ]
  [0.   0.  ]]

 [[0.13 0.43]
  [0.   0.01]
  [0.   0.  ]
  [0.   0.01]
  [0.   0.  ]
  [0.   0.  ]]

 [[0.   0.04]
  [0.   0.03]
  [0.   0.  ]
  [0.21 0.31]
  [0.   0.  ]
  [0.   0.  ]]

 [[0.   0.03]
  [0.   0.04]
  [0.2  0.35]
  [0.   0.  ]
  [0.   0.  ]
  [0.   0.  ]]

 [[0.   0.03]
  [0.   0.04]
  [0.   0.  ]
  [0.   0.01]
  [0.   0.  ]
  [0.22 0.24]]

 [[0.   0.03]
  [0.   0.03]
  [0.   0.  ]
  [0.   0.  ]
  [0.23 0.24]
  [0.   0.  ]]]
Alphas: 
[0.01 0.11]

 Baseline: 
[0.00276816 0.00250772 0.00385232 0.00345824 0.00241811 0.00246908]


,Endogeneity,Likelihood,Max KS Test,Max LB Test,Min ED Statistic
0,0.577573,0.02271,2.232886e-07,1.161099e-63,15.90674


# Expectation Maximization

In [249]:
EMLearner = HawkesEM(10, verbose=False, tol=1e-5)

EMLearner.fit(list(all_timestamps))


In [250]:
residuals = goodness_of_fit_nonpar(EMLearner, all_timestamps, 1, integrate.trapz)
print()

Kernel [0, 0] set.
Kernel [0, 1] set.
Kernel [0, 2] set.
Kernel [0, 3] set.
Kernel [0, 4] set.
Kernel [0, 5] set.
Kernel [1, 0] set.
Kernel [1, 1] set.
Kernel [1, 2] set.
Kernel [1, 3] set.
Kernel [1, 4] set.
Kernel [1, 5] set.
Kernel [2, 0] set.
Kernel [2, 1] set.
Kernel [2, 2] set.
Kernel [2, 3] set.
Kernel [2, 4] set.
Kernel [2, 5] set.
Kernel [3, 0] set.
Kernel [3, 1] set.
Kernel [3, 2] set.
Kernel [3, 3] set.
Kernel [3, 4] set.
Kernel [3, 5] set.
Kernel [4, 0] set.
Kernel [4, 1] set.
Kernel [4, 2] set.
Kernel [4, 3] set.
Kernel [4, 4] set.
Kernel [4, 5] set.
Kernel [5, 0] set.
Kernel [5, 1] set.
Kernel [5, 2] set.
Kernel [5, 3] set.
Kernel [5, 4] set.
Kernel [5, 5] set.
0
1
2
3
4
5



In [251]:
# alphas = 
# betas = EMLearner.decays
baseline = EMLearner.baseline
loglik = EMLearner.score(list(all_timestamps))

# print("Alphas: ")
# print(np.round(alphas, 2))

print("\n Baseline: ")
print(baseline)

table = {
    "Endogeneity": [max(np.linalg.eigvals(EMLearner.get_kernel_norms()))],
    "Likelihood": [loglik],
    "Max KS Test": max([ks.pvalue for ks in ks_test(residuals)]),
    "Max LB Test": max([list(lb.lb_pvalue)[0] for lb in lb_test(residuals)]),
    "Min ED Statistic": min(ed_test(residuals))
}

df = pd.DataFrame(table)
df


 Baseline: 
[0.00219566 0.00210803 0.0032169  0.00233925 0.00417986 0.00422816]


,Endogeneity,Likelihood,Max KS Test,Max LB Test,Min ED Statistic
0,0.601402,126.860975,0.073802,0.429189,1.347567


# Fitting Single TS

In [41]:
# session = "AM"
# session = "MID"
session = "PM"

n_minus_ts_ticks_60dc = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_60dc.csv')
n_minus_ts_ticks_60dp = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_60dp.csv')
n_minus_ts_ticks_50dc = csv_reader(f'final_dataset/{session}/plus_minus/n_minus_ts_ticks_50dc.csv')

n_plus_ts_ticks_60dc = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_60dc.csv')
n_plus_ts_ticks_60dp = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_60dp.csv')
n_plus_ts_ticks_50dc = csv_reader(f'final_dataset/{session}/plus_minus/n_plus_ts_ticks_50dc.csv')

In [42]:
data_60dc = [
    n_plus_ts_ticks_60dc, n_minus_ts_ticks_60dc,
]

data_50d = [
    n_plus_ts_ticks_50dc, n_minus_ts_ticks_50dc,
]

data_60dp = [
    n_plus_ts_ticks_60dp, n_minus_ts_ticks_60dp,
]

# all_data = [
#             n_plus_ts_ticks_25dc, n_minus_ts_ticks_25dc,
#             n_plus_ts_ticks_50dc, n_minus_ts_ticks_50dc,
#             n_plus_ts_ticks_25dp, n_minus_ts_ticks_25dp,
# ]

def convert_raw_to_timestamps(raw_data_lst):
    all_timestamps = [
        np.array(list(df['Bid_time']))
        for df in raw_data_lst
    ]

    max_min_time = max([min(ts_df) for ts_df in all_timestamps])
    min_max_time = min([max(ts_df) for ts_df in all_timestamps])

    trimmed_ts = [
        ts[(max_min_time < ts) & (ts < min_max_time)] - max_min_time
        for ts in all_timestamps
    ]

    for ts_ in trimmed_ts:
        print(ts_[0])
        print("Increasing: {}".format(np.all(np.diff(ts_) > 0)))
    
    return trimmed_ts

timestamps_60dc = convert_raw_to_timestamps(data_60dc)
timestamps_60dp = convert_raw_to_timestamps(data_60dp)
timestamps_50d = convert_raw_to_timestamps(data_50d)


7.0
Increasing: True
4.0
Increasing: True
2.0
Increasing: True
3.0
Increasing: True
192.0
Increasing: True
194.0
Increasing: True


In [81]:
start = 0.05
stop = 1.5

single_expo_60dc_best_beta = grid_search(exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=timestamps_60dc)
single_expo_60dp_best_beta = grid_search(exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=timestamps_60dp)
single_expo_50d_best_beta = grid_search(exp_minimiser, start = start, stop = stop, tol=1e-7, timestamps=timestamps_50d)

Grid: 0.05 : 1.5
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Step equals 0... at 0
Best value so f

In [110]:
single_expo_60dc_learner = HawkesExpKern(decays=single_expo_60dc_best_beta[0], tol=1e-10, penalty='elasticnet', elastic_net_ratio=0.9, max_iter=2000)
single_expo_60dp_learner = HawkesExpKern(decays=single_expo_60dp_best_beta[0], tol=1e-10, penalty='elasticnet', elastic_net_ratio=0.9, max_iter=2000)
single_expo_50d_learner = HawkesExpKern(decays=single_expo_50d_best_beta[0], tol=1e-10, penalty='elasticnet', elastic_net_ratio=0.9, max_iter=2000)

single_expo_60dc_learner.fit(timestamps_60dc)
single_expo_60dp_learner.fit(timestamps_60dp)
single_expo_50d_learner.fit(timestamps_50d)
# residuals = goodness_of_fit_par(single_expo_learner, all_timestamps, 1, integrate.trapz)


In [166]:
all_ts = []
all_ts.extend(timestamps_60dp)
all_ts.extend(timestamps_50d)
all_ts.extend(timestamps_60dc)
max_time = max([len(ts) for ts in all_ts])
max_time

10485

In [167]:
all_ts

[array([6.500000e+02, 5.842000e+03, 9.115000e+03, ..., 1.197753e+06,
        1.197801e+06, 1.197856e+06]),
 array([4.010000e+02, 2.183000e+03, 2.361000e+03, ..., 1.197756e+06,
        1.197809e+06, 1.197918e+06]),
 array([3.300000e+01, 5.600000e+01, 7.600000e+01, ..., 1.180877e+06,
        1.180886e+06, 1.180897e+06]),
 array([4.800000e+01, 5.200000e+01, 8.500000e+01, ..., 1.180865e+06,
        1.180884e+06, 1.180889e+06]),
 array([1.600000e+01, 6.300000e+01, 7.900000e+01, ..., 1.199527e+06,
        1.199536e+06, 1.199589e+06]),
 array([2.800000e+01, 3.200000e+01, 6.600000e+01, ..., 1.199526e+06,
        1.199537e+06, 1.199595e+06])]

In [175]:
def goodness_of_fit_nonpar(nplearner, arrivals, timestep, method):
    # setup simulation object
    hp = SimuHawkes(baseline=nplearner.baseline)
    # set kernels
    support = np.arange(0, np.max(nplearner.get_kernel_supports()), timestep)

    for i, j in itertools.product(range(nplearner.n_nodes), repeat=2):
        print('Kernel {} set.'.format([i, j]))
        y = nplearner.get_kernel_values(i, j, support)
        kernel = HawkesKernelTimeFunc(t_values=support, y_values=y)
        hp.set_kernel(i, j, kernel)

    hp.track_intensity(timestep)
    hp.set_timestamps(timestamps=arrivals)
    dimension = hp.n_nodes
    intensities = hp.tracked_intensity
    x = hp.intensity_tracked_times
    residuals = [resid(x, intensities, arrivals, dim, method) for dim in range(dimension)]
    return residualsed_times
    residuals = [resid(x, intensities, arrivals, dim, method) for dim in range(dimension)]
    return residuals

In [155]:
timestep = 1

learners = [
    single_expo_60dp_learner, single_expo_50d_learner, single_expo_60dc_learner
]

baselines = [
    single_expo_60dp_learner.baseline[0], single_expo_60dp_learner.baseline[1],
    single_expo_50d_learner.baseline[0], single_expo_50d_learner.baseline[1],
    single_expo_60dc_learner.baseline[0], single_expo_60dc_learner.baseline[1],
]
print(baselines)

max_support = np.arange(0, np.max([np.max(hp.get_kernel_supports()) for hp in learners]), timestep)
print(max_support)

double_0_kernel = [HawkesKernel0(), HawkesKernel0()]

combined_hp = SimuHawkes(baseline=baselines, verbose=True)

lines = []
for idx, learner in enumerate(learners):
    line_1, line_2 = [], []
    for i, j in itertools.product(range(learner.n_nodes), repeat=2):
        y = learner.get_kernel_values(i, j, max_support)
        kernel = HawkesKernelTimeFunc(t_values=max_support, y_values=y)
        if i == 0:
            line_1.append(kernel)
        else:
            line_2.append(kernel)
    
    if idx == 0:
        line_1 = line_1 + double_0_kernel + double_0_kernel
        line_2 = line_2 + double_0_kernel + double_0_kernel
    elif idx == 1:
        line_1 = double_0_kernel + line_1 + double_0_kernel
        line_2 = double_0_kernel + line_2 + double_0_kernel
    else:
        line_1 = double_0_kernel + double_0_kernel + line_1
        line_2 = double_0_kernel + double_0_kernel + line_2
        
    lines.append(line_1)
    lines.append(line_2)
    
for i in range(len(lines)):
    for j in range(len(lines[0])):
        combined_hp.set_kernel(i, j, lines[i][j])

combined_hp.end_time = max_time

dt = 1
# hawkes.track_intensity(dt)
combined_hp.simulate()

[0.002511165442287683, 0.002240988346064782, 0.004274171786247918, 0.0037695340699965855, 0.002955776823588191, 0.0027863065876883868]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78.]
----------------------------------------
Launching simulation using SimuHawkes...
Done simulating using SimuHawkes in 4.18e-03 seconds.


In [177]:
combined_hp.track_intensity(timestep)
combined_hp.set_timestamps(timestamps=all_ts)
dimension = combined_hp.n_nodes
intensities = combined_hp.tracked_intensity
x = combined_hp.intensity_tracked_times
residuals = [resid(x, intensities, all_ts, dim, integrate.trapz) for dim in range(dimension)]
residuals

0
1
2
3
4
5


[array([14.67947122,  9.26282698,  1.02300258, ...,  0.34825962,
         0.963247  ,  0.73485949]),
 array([4.54327516, 0.39889593, 6.79691765, ..., 0.08547594, 0.71863903,
        1.20591707]),
 array([0.38736616, 0.58894309, 1.65640229, ..., 0.06689435, 0.2066223 ,
        0.52218158]),
 array([0.06426768, 0.96704888, 0.53549665, ..., 1.11704554, 0.67509897,
        0.29840096]),
 array([1.06976606, 0.46611362, 0.11241026, ..., 0.20186149, 0.55181744,
        1.05252501]),
 array([0.07660173, 0.39932012, 0.46161568, ..., 0.09569112, 0.697756  ,
        1.19890137])]

In [183]:
# alphas = combined_hp.adjacency
# betas = combined_hp.decays
baseline = combined_hp.baseline
# loglik = combined_hp.score()

# print("Alphas: ")
# print(np.round(alphas, 2))


# print("Alphas: ")
# print(np.round(betas, 2))

print("\n Baseline: ")
print(baseline)

table = {
#     "Endogeneity": [max(np.linalg.eigvals(alphas.sum(-1)))],
#     "Likelihood": [loglik],
    "Max KS Test": max([ks.pvalue for ks in ks_test(residuals)]),
    "Max LB Test": max([list(lb.lb_pvalue)[0] for lb in lb_test(residuals)]),
    "Min ED Statistic": min(ed_test(residuals))
}

df = pd.DataFrame(table, index=[0])
df


 Baseline: 
[0.00251117 0.00224099 0.00427417 0.00376953 0.00295578 0.00278631]


,Max KS Test,Max LB Test,Min ED Statistic
0,2.651634e-11,4.371330e-77,18.131002
